In [63]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv("housing.csv")

# Remove rows with missing values
df = df.dropna()

# Convert the object column to str (if needed)
df['ocean_proximity'] = df['ocean_proximity'].astype(str)

# Replace all occurrences of a word with another word
df['ocean_proximity'] = df['ocean_proximity'].str.replace('NEAR BAY', '01', regex=True)
df['ocean_proximity'] = df['ocean_proximity'].str.replace('<1H OCEAN', '02', regex=True)
df['ocean_proximity'] = df['ocean_proximity'].str.replace('INLAND', '03', regex=True)
df['ocean_proximity'] = df['ocean_proximity'].str.replace('NEAR OCEAN', '04', regex=True)
df['ocean_proximity'] = df['ocean_proximity'].str.replace('ISLAND', '05', regex=True)

# Convert an object column to str
df['ocean_proximity'] = df['ocean_proximity'].astype(object).astype(float)

# Write the updated DataFrame to a new CSV file
df.to_csv('final.csv', index=False)

In [64]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('final.csv')

# Check for empty or NA values
missing_values = df.isnull().sum().sum()

if missing_values > 0:
    print("There are missing values in the CSV file.")
else:
    print("There are no missing values in the CSV file.")

There are no missing values in the CSV file.


In [65]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Read the CSV file into a DataFrame
df = pd.read_csv('final.csv')

# Split the data into input features (X) and target variable (y)
X = df.drop('ocean_proximity', axis=1)  # Input features
y = df['ocean_proximity']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the Random Forest regression model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model using mean squared error
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

Mean Squared Error: 0.04947722045510154


In [66]:
Mean Squared Error: 4509443326.343708
Mean Squared Error: 2408424402.3731775
Mean Squared Error: 2468696938.732993
------
Mean Squared Error: 0.04947722045510154
Mean Squared Error: 0.04947722045510154
Mean Squared Error: 0.04947722045510154


SyntaxError: invalid syntax (1966666092.py, line 1)

In [69]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes
import csv

# Generate a key pair
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048
)
public_key = private_key.public_key()

# Save the private key to a file
with open('private_key.pem', 'wb') as f:
    f.write(private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    ))

# Save the public key to a file
with open('public_key.pem', 'wb') as f:
    f.write(public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    ))

# Load the CSV file
with open('final.csv', 'r') as f:
    reader = csv.reader(f)
    data = list(reader)

# Encrypt the data
encrypted_data = []
for row in data:
    encrypted_row = []
    for cell in row:
        encrypted_cell = public_key.encrypt(
            bytes(cell, 'utf-8'),
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
        encrypted_row.append(encrypted_cell)
    encrypted_data.append(encrypted_row)

# Save the encrypted data to a new file
with open('enc_final.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(encrypted_data)


In [73]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa, padding
import csv

# Load the private key
with open('private_key.pem', 'rb') as f:
    private_key = serialization.load_pem_private_key(
        f.read(),
        password=None
    )

# Load the encrypted data
encrypted_data = []
with open('enc_final.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        encrypted_row = []
        for cell in row:
            encrypted_row.append(eval(cell))
        encrypted_data.append(encrypted_row)

# Decrypt the data
decrypted_data = []
for row in encrypted_data:
    decrypted_row = []
    for cell in row:
        decrypted_cell = private_key.decrypt(
            cell,
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
        decrypted_row.append(decrypted_cell.decode('utf-8'))
    decrypted_data.append(decrypted_row)

# Save the decrypted data to a new file
with open('dec_final.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(decrypted_data)